In [1]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

'\n掛載Google硬碟\n安裝套件\n引用套件\nAPP應用準備\n消息準備\nhandler執行方法設計\n啟動應用\n\n'

In [2]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
'''

流程解析

'''


'\n\n流程解析\n\n'

In [4]:
!pip install line-bot-sdk flask flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [6]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
# 跟line 溝通用
line_bot_api = LineBotApi("vtaRo4pkLqIZjk50gBJ53fFtMpg+8unl4TvLKbyvENmbWkzdriMNIeVm87ZWRBdTeDL7YM4hGte+BrNMmYscJ1WNPvCQbheXUbVevrlKB3zP3CaeRjbPQRPHMsqW+037bLOogqjOrh5ssRj53kqQMQdB04t89/1O/w1cDnyilFU=")
# 收發訊息用
handler = WebhookHandler("de385af73d2121e65ca6fc0490991b02")

In [7]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [30]:
'''

準備QuickReply的Button
QuickReply 快速回復
有很多小按鍵,按鍵使用後消失
準備準備QuickReply 必須先把按鍵準備好

'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 

## 點擊後，以用戶身份發送文字消息
## MessageAction
"""
QuickReplyButton
action >> 填 action
MessageAction 以用戶發文字
label 按鍵的文字
text 以用戶角度講話
""" 
textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="不是很開心喔", 
        text="你怎麼敢跟我這樣說話!"
    )
)

## 點擊後，彈跳出選擇時間之視窗
## DatetimePickerAction
"""
DatetimePickerAction
label 按鍵的文字
data 隱藏資訊給用戶
mode date 給用戶輸入的畫面是天數
""" 
dateQuickReplyButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="時間選擇",
        data="data3",                       
        mode="date"
    )
)


## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="拍照")
)

## 點擊後，切換至照片相簿選擇
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

## 點擊後，跳出地理位置
# GPS地理位子
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)

## 點擊後，以Postback事件回應Server 
"""
利用自己設計的Data1 做很多特殊功能
"""
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我是PostbackEvent", data="Data1")
)

In [24]:
'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
# QuickReplyButton的參數 items 通常為一個list
# list 裡面放那些buttons
quickReplyList = QuickReply(
    items = [textQuickReplyButton, dateQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, locationQRB,postbackQRB]
)

In [25]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
# quick reply 通常必須夾帶在 xxxSendMessage的裡面
# xxxxxSendMessage
# 都會有quick reply參數 此參數就是放在QuickReply
quick_reply_text_send_message = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

In [26]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 根據自定義菜單四張故事線的圖，設定相對應image
# 用戶輸入@reply
template_message_dict = {
  "@reply":quick_reply_text_send_message,
}

In [27]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理
from linebot.models import PostbackEvent

# Postback很重要 結合QuuckReply 可以做很多好玩的不可逆消息
@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，我們會派專家您聯繫'
                )
            )
    else:
        pass

In [28]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
# 為了要觸發字典裡的消息 設計當用戶發文字 就會到字典裡找消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@reply")
        )

In [29]:
# 主程序運行
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://7b63-35-243-189-45.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"message","message":{"type":"text","id":"17811108493675","text":"還好"},"webhookEventId":"01GVMH02B2R71RA8MEYCV3RED5","deliveryContext":{"isRedelivery":false},"timestamp":1678947387239,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"2a74991f9562464da30d3030109257be","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 06:16:29] "POST / HTTP/1.1" 200 -
ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.9/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-7-76ebdfe53b31>", line 27, in callback
    handler.handle(body, signature)
  File "/usr/local/lib/python3.9/dist-packages/linebot/webhook.py", line 263, in handle
    self.__invoke_func(func, event, payload)
  File "/usr/local/lib/python3.9/dist-packages/linebot/

{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"message","message":{"type":"text","id":"17811109245119","text":"@more"},"webhookEventId":"01GVMH0BEH3ZV4WKSAY9E1TJ5B","deliveryContext":{"isRedelivery":false},"timestamp":1678947396773,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"2d8c748eead6475699e162341be4cffc","mode":"active"}]}
{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"message","message":{"type":"text","id":"17811110165228","text":"@reply"},"webhookEventId":"01GVMH0Q63879T2WRRBH2S07CT","deliveryContext":{"isRedelivery":false},"timestamp":1678947408707,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"f2b6153e88ce4436812f4a95c8052463","mode":"active"}]}


...

INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 06:18:27] "POST / HTTP/1.1" 200 -


{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"postback","postback":{"data":"data3","params":{"date":"2023-03-18"}},"webhookEventId":"01GVMH3WG3N5TVX2717JVHYMGE","deliveryContext":{"isRedelivery":false},"timestamp":1678947512624,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"b642c3d9a7384f51b565cd497656992d","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 06:18:33] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 06:18:58] "POST / HTTP/1.1" 200 -


{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"message","message":{"type":"image","id":"17811120154120","contentProvider":{"type":"line"}},"webhookEventId":"01GVMH4N9GWQQ01MEFD52MF0RM","deliveryContext":{"isRedelivery":false},"timestamp":1678947538062,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"5dd5eb01c2894d2784deb7224bbd72c6","mode":"active"}]}
{"destination":"U54dcf9ab5f592860405fcec0d6979997","events":[{"type":"message","message":{"type":"text","id":"17811120894043","text":"@reply"},"webhookEventId":"01GVMH4Y3N2263E944P5T29AZA","deliveryContext":{"isRedelivery":false},"timestamp":1678947546754,"source":{"type":"user","userId":"Ua19163cfb927dce013b280948e91caf0"},"replyToken":"704040c8cfc64377acffe7511220c8e0","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 06:19:08] "POST / HTTP/1.1" 200 -
